<a href="https://colab.research.google.com/github/cmfritz/capstone/blob/main/data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Github/capstone

/content/gdrive/My Drive/Github/capstone


In [ ]:
#import packages
import pandas as pd
import numpy as np
import gc

In [ ]:
# import my custom library into colab
import sys
sys.path.append('/content/gdrive/MyDrive/Github/capstone/')
import mylibrary as mylib

First, get the sector (classification) data for each patent.

In [ ]:
# Load data directly from USPTO 
# Data contains patent no. and its assigned sector id
url = 'https://s3.amazonaws.com/data.patentsview.org/download/wipo.tsv.zip'
df_field_lookup = mylib.tsv_to_df(url)
df_field_lookup

,patent_id,field_id,sequence
0,10001059,31,1
1,10001517,10,1
2,10002233,6,0
3,10002234,13,2
4,10002235,6,1
...,...,...,...
10296072,9999997,26,0
10296073,9999997,29,1
10296074,9999998,29,0
10296075,9999998,21,1


In [ ]:
# remove duplicate entries
df_field_lookup = df_field_lookup.drop_duplicates()

In [ ]:
# Load tsv file that explains the id and sector relationship
url = 'https://s3.amazonaws.com/data.patentsview.org/download/wipo_field.tsv.zip'
df_field_key = mylib.tsv_to_df(url)
df_field_key

,id,sector_title,field_title
0,1,Electrical engineering,"Electrical machinery, apparatus, energy"
1,10,Instruments,Measurement
2,11,Instruments,Analysis of biological materials
3,12,Instruments,Control
4,13,Instruments,Medical technology
...,...,...,...
65,D30,Design,Articles for the care and handling of animals
66,D31,Design,Machines and appliances for preparing food or ...
67,D32,Design,"Graphic symbols and logos, surface patterns, o..."
68,D34,Design,Material or Article Handling Equipment


In [ ]:
# Merge dataframes to match 'field_id' with the sector title
df_pats_w_sectors = df_field_lookup.merge(df_field_key, 
                                          left_on='field_id', 
                                          right_on='id')
df_pats_w_sectors


,patent_id,field_id,sequence,id,sector_title,field_title
0,10001059,31,1,31,Mechanical engineering,Mechanical elements
1,10053304,31,2,31,Mechanical engineering,Mechanical elements
2,10085485,31,3,31,Mechanical engineering,Mechanical elements
3,10138340,31,3,31,Mechanical engineering,Mechanical elements
4,10189524,31,3,31,Mechanical engineering,Mechanical elements
...,...,...,...,...,...,...
9999995,10818791,22,1,22,Chemistry,Micro-structural and nano-technology
9999996,10818792,22,1,22,Chemistry,Micro-structural and nano-technology
9999997,10818805,22,1,22,Chemistry,Micro-structural and nano-technology
9999998,10818835,22,2,22,Chemistry,Micro-structural and nano-technology


In [ ]:
# drop unnecessary columns
df_pats_w_sectors = df_pats_w_sectors.drop(['field_id', 'sequence', 'id'], 
                                           axis=1)
df_pats_w_sectors

,patent_id,sector_title,field_title
0,10001059,Mechanical engineering,Mechanical elements
1,10053304,Mechanical engineering,Mechanical elements
2,10085485,Mechanical engineering,Mechanical elements
3,10138340,Mechanical engineering,Mechanical elements
4,10189524,Mechanical engineering,Mechanical elements
...,...,...,...
9999995,10818791,Chemistry,Micro-structural and nano-technology
9999996,10818792,Chemistry,Micro-structural and nano-technology
9999997,10818805,Chemistry,Micro-structural and nano-technology
9999998,10818835,Chemistry,Micro-structural and nano-technology


In [ ]:
# Store as pickle for using in another notebook
df_pats_w_sectors.to_pickle('data/pats_w_sectors.pkl')

Now, get the claims for each patent, which is a large amount of data. We will pair this down to the 1st claim, which is typically the most important.

In [ ]:
def clean_tsv(df, cols2drop):
  df = df.drop(cols2drop, axis=1)

  # get rid of reissue, design, and plant patents
  df = df[df.patent_id.str.isnumeric()==True]

  # make sure dtype for patid is integer
  df.patent_id = df.patent_id.astype('int64')

  # keep only rows with 1st claim, then drop column
  df = df[df.claim_number=='00001']
  df = df.drop('claim_number', axis=1)

  return df

In [ ]:
# Load 2020 data
url = 'https://s3.amazonaws.com/data.patentsview.org/claims/claims_2020.tsv.zip'
df = mylib.tsv_to_df(url)
df.head()

,uuid,patent_id,text,dependent,sequence,num,exemplary
0,87720c54-7d3e-11ea-83c3-121df0c29c1e,10524402,1. A method comprising: capturing an image of ...,NaN,0,00001,1
1,87720dc3-7d3e-11ea-83c3-121df0c29c1e,10524402,"2. The method of claim 1 , wherein identifying...",claim 1,1,00002,0
2,877214e9-7d3e-11ea-83c3-121df0c29c1e,10524402,"3. The method of claim 1 , wherein the propert...",claim 1,2,00003,0
3,87721630-7d3e-11ea-83c3-121df0c29c1e,10524402,"4. The method of claim 1 , wherein selecting a...",claim 1,3,00004,0
4,87721730-7d3e-11ea-83c3-121df0c29c1e,10524402,"5. The method of claim 1 , wherein the propert...",claim 1,4,00005,0


In [ ]:
# Need to process 2020 data differently from other data since it is formatted 
# differently from other years

# rename num column since num name causes issues
df = df.rename(columns={'num':'claim_number'})

# use my cleaning function to clean up df
cols2drop=['uuid', 'sequence', 'exemplary', 'dependent']
df = clean_tsv(df, cols2drop)

# start master dataframe to collect data for all years
df_master = df
del df
gc.collect()
df_master

,patent_id,text
0,10524402,1. A method comprising: capturing an image of ...
24,10524403,1. A method for determining a weight of a payl...
29,10524404,"1. A break-away assembly, comprising: a swivel..."
49,10524405,"1. An agricultural implement system, comprisin..."
59,10524406,1. A hitch comprising: a tractor; a towed or s...
...,...,...
5888339,10881037,1. A method of manufacturing a printed circuit...
5888358,10881038,"1. A control system for a motor vehicle, in pa..."
5888378,10881039,1. An Electromagnetic Interference shielding f...
5888398,10881040,"1. An adjustable mobile cart, including: at le..."


In [ ]:
# Iterate to load 2011 - 2019 to round out 10 years of data
years = range(2011,2020,1)
cols2drop = ['uuid', 'sequence', 'exemplary', 'dependent', 
             'patent_date','created_date', 'updated_date']
for year in years:
  url = f'https://s3.amazonaws.com/data.patentsview.org/claims/claims_{year}.tsv.zip'
  df = clean_tsv(mylib.tsv_to_df(url), cols2drop) # download and clean
  df_master = df_master.append(df, ignore_index=True) # append to master df
  del df
  gc.collect()

In [ ]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001035 entries, 0 to 3001034
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   patent_id  int64 
 1   text       object
dtypes: int64(1), object(1)
memory usage: 45.8+ MB


In [ ]:
# Store as pickle for using in another notebook
df_master.to_pickle('data/master_data.pkl')